# Chains (LangChain v1.2)

**LCEL(LangChain Expression Language)**을 사용해서 모든 구성 요소가 `Runnable` 인터페이스로 통합되어 파이프라인(`|`)으로 연결될 수 있다.

```python
chain = prompt | model | output_parser  # 기본 구조
```

**구성 요소 업데이트 (v1.2 기준)**
1. **PromptTemplate**  
   - `Runnable`로 변환되어 LCEL 파이프라인에 직접 통합  
   ```python
   prompt = ChatPromptTemplate.from_template("...")
   ```

2. **LLM/ChatModel**  
   - `ChatOpenAI`, `ChatAnthropic` 등이 `Runnable` 구현  
   ```python
   model = ChatOpenAI(model="gpt-4o")
   ```

3. **Memory**  
   - `RunnableWithMessageHistory`로 통합 관리 (또는 LangGraph Persistence 사용)
   ```python
   chain_with_memory = RunnableWithMessageHistory(
       base_chain,
       get_session_history
   )
   ```

4. **Output Parsers**  
   - `StrOutputParser()`, `JsonOutputParser()` 등이 `Runnable`로 작동  
   ```python
   output_parser = JsonOutputParser()
   ```

5. **Tools**  
   - `@tool` 데코레이터로 생성 후 `RunnableLambda`로 변환  
   ```python
   @tool
   def search(query: str) -> str: ...
   ```

**체인 유형별 구현**


1. Simple Chain  

    ```python
    chain = prompt | model | output_parser
    response = chain.invoke({"input": "..."})
    ```

2. Sequential Chain  

    ```python
    chain = (
        {"step1_output": prompt1 | model1}  # 첫 번째 체인 결과 매핑
        | prompt2
        | model2
    )
    ```

3. Conditional Chain
    - `RunnableBranch` 사용

    ```python
    branch = RunnableBranch(
        (lambda x: x["topic"] == "math", math_chain),
        (lambda x: x["topic"] == "history", history_chain),
        default_chain
    )
    ```

4. Memory Chain  

    ```python
    memory_chain = RunnableWithMessageHistory(
        core_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="history"
    )
    ```


**🚨 v1.2 주요 변경점**

- **Legacy Chain 클래스 완전 폐기**: `LLMChain`, `SequentialChain` 등은 `langchain-classic`으로 이동되거나 삭제됨 → `Runnable` (LCEL)로 통합
- **에이전트 통합**: `create_agent` (LangGraph 기반)가 표준

In [1]:
%pip install langchain langchain-openai langchain-community -Uq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://eu.api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록

## Simple Chain

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template('{country}의 수도는 어디인가요?')
llm = init_chat_model('gpt-4.1-mini')
output_parser = StrOutputParser()       # 응답을 최종 문자열로 변환

chain = prompt | llm | output_parser    # chain : 프롬프트 -> LLM -> 파서
print(chain.invoke(input={'country': '대한민국'}))  # dict 형태로 변수를 주입해서 실행
print(chain.invoke("대한민국"))                     # 프롬프트 템플릿 변수가 2개 이상인 경우 오류가 발생할 수 있다.

대한민국의 수도는 서울특별시입니다.
대한민국의 수도는 서울특별시입니다.


## Sequential Chain

In [5]:
prompt1 = PromptTemplate.from_template('다음 내용을 한글로 번역하세요. \n\n{eng_text}')
prompt2 = PromptTemplate.from_template('다음 내용을 요약하세요. \n\n{kor_text}')

llm = init_chat_model('gpt-4.1-mini')
output_parser = StrOutputParser()

# 번역 체인
chain1 = prompt1 | llm

eng_text = """
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""
print(chain1.invoke(eng_text))

# 요약 체인
chain2 = prompt2 | llm | output_parser

kor_text = """
LLM의 한 가지 한계는 특정 문서나 이메일과 같은 맥락 정보를 갖고 있지 않다는 점입니다. 이를 해결하려면 LLM이 특정 외부 데이터에 접근할 수 있도록 해야 합니다.
이를 위해서는 먼저 문서 로더를 사용해 외부 데이터를 불러와야 합니다. LangChain은 PDF, 이메일부터 웹사이트, 유튜브 영상에 이르기까지 다양한 유형의 문서를 위한 여러 종류의 로더를 제공합니다.
"""
print(chain2.invoke(kor_text))

content='LLM의 한 가지 한계는 특정 문서나 이메일과 같은 맥락 정보를 갖고 있지 않다는 점입니다. 이를 해결하려면 LLM에게 특정 외부 데이터에 대한 접근 권한을 부여하면 됩니다.  \n이를 위해서는 먼저 문서 로더를 사용해 외부 데이터를 불러와야 합니다. LangChain은 PDF, 이메일부터 웹사이트, 유튜브 동영상에 이르기까지 다양한 유형의 문서를 위한 여러 가지 로더를 제공합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 98, 'total_tokens': 202, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_e01c6f58e1', 'id': 'chatcmpl-D5kZTnVKBuGg88GSCyKcxibnKQVtr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c2be6-2c1d-7173-b45b-604731dfcf54-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 98, 'output_tokens': 104, 'total_tokens': 202, 'input_token_details': {'audio': 0, 

In [7]:
# 통합체인
chain = chain1 | chain2
chain.invoke({"eng_text":eng_text})

'LLM의 한계 중 하나는 특정 문서나 이메일 등 맥락 정보에 접근할 수 없다는 점이며, 이를 극복하기 위해 LLM에 외부 데이터 접근 권한을 부여할 수 있다. 이를 위해 문서 로더를 사용해 외부 데이터를 불러오는데, LangChain은 PDF, 이메일, 웹사이트, 유튜브 영상 등 다양한 문서 유형에 맞는 로더를 제공한다.'

## Conditional Chain

In [ ]:
from langchain_core.runnables import RunnableBranch # 조건에 따라 체인을 분기 실행하는 Runnable

llm = init_chat_model('openai:gpt-4.1-mini')

# 수학 체인
math_prompt = PromptTemplate.from_template(
    '다음 문제를 풀어주세요. 단계적인 풀이를 수식(LaTex)과 함께 작성해주세요. \n\n{question}'
)
math_chain = math_prompt | llm | output_parser

# 기본 체인 
default_prompt = PromptTemplate.from_template(
    '당신은 친절하고, 감성적인 공감능력이 좋은 챗봇입니다. 다음 질문에 답변해주세요. \n\n{question}'
)

default_chain = default_prompt | llm | output_parser    # math_prompt -> llm -> 문자열 파싱

# math_chain 선택 함수 (math_chain을 사용하는 경우 True 반환)
def is_math_question(input_dict:dict) -> bool:
    """질문에 '계산 '또는 calc가 포함되면 수학 체인을 선택한다."""
    question: str = input_dict.get('question','')   # 입력 dict에서 question 추출(없으면 빈문자)
    return '계산' in question or 'calc' in question

# 분기 체인
branch_chain = RunnableBranch(
    (is_math_question, math_chain), # 조건이 Ture면 math_chain 실행
    default_chain                   # 그 외에는 default_chain 실행
)

branch_chain.invoke({'question': '125 * 3 + 50 이거를 계산해줘'})

'물론입니다! 문제를 단계별로 계산해 보겠습니다.\n\n주어진 식은:\n\\[\n125 \\times 3 + 50\n\\]\n\n### 1단계: 곱셈부터 계산하기\n곱셈을 먼저 수행합니다.\n\\[\n125 \\times 3 = 375\n\\]\n\n### 2단계: 덧셈 수행하기\n곱셈 결과에 50을 더합니다.\n\\[\n375 + 50 = 425\n\\]\n\n### 최종 답:\n\\[\n\\boxed{425}\n\\]\n\n따라서 \\(125 \\times 3 + 50 = 425\\) 입니다.'

In [9]:
branch_chain.invoke({"question" : "나 오늘 우울해. 빵 사줘."})

'오늘 많이 힘들었나 봐요. 그런 날엔 작은 위로가 큰 힘이 되죠. 제가 직접 빵을 사줄 순 없지만, 함께 빵 맛있는 곳 이야기해볼까요? 어떤 빵 좋아해요? 혹은 그냥 마음을 달래줄 따뜻한 이야기라도 들려드릴게요. 혼자서 힘들어하지 말고, 언제든 이야기 나누어요. 😊'

## Memory chain

`RinnableWithMessageHistory`를 사용해서 대화내역을 기억하는 chian을 생성한다.

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory              # LangChain 대화기록 인터페이스
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage    # 메시지 타입
from pydantic import BaseModel, Field   # Pydantic 모델(검증/기본값) 도구
from typing import List

# 세션별 메시지 저장소 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """사용자별(세션) 대화내역을 기록하는 클래스"""
    messages: List[BaseMessage] = Field(default_factory=list)   # 인스턴스마다 독립적인 message list를 생성
    
    # 전달받은 메시지들을 기존 리스트 뒤에 추가
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
        
    # 저장된 메시지를 모두 초기화
    def clear(self) -> None:
        self.messages = []

store= {}   # 세션 ID(key) -> 히스토리 객체(value)를 담는 전역 저장소(dict)

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    """session_id에 해당하는 대화내역 객체를 반환(없으면 생성)"""
    if session_id not in store: # 아직 해당 세션의 히스토리가 없으면
        store[session_id] = InMemoryHistory()   # 새 대화내역 히스토리 객체 생성해서 store에 저장
    return store[session_id]    # 해당 세션의 히스토리 객체 반환

history1 = get_by_session_id('1')   # 세션ID '1의 히스토리 가져옴 (없으면 생성)
history1.add_messages([[AIMessage(content='반갑습니다. 홍길동님')]])
history1.add_messages([[HumanMessage(content='그래 나 홍길동이야~ 반갑다!')]])
print(f'{history1 = }') # history1 객체 출력

history2= get_by_session_id('2')
print(f'{history2 = }')

history1 = InMemoryHistory(messages=[[AIMessage(content='반갑습니다. 홍길동님', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])], [HumanMessage(content='그래 나 홍길동이야~ 반갑다!', additional_kwargs={}, response_metadata={})]])
history2 = InMemoryHistory(messages=[])


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # 채팅 프롬프르 템플릿 / 히스토리 자리표시자
from langchain_core.runnables import RunnableWithMessageHistory             # 실행시 히스토리를 붙여주는 Runnable

prompt = ChatPromptTemplate.from_messages([
    ('system','당신은 {domain}분야의 전문가 챗봇입니다.'),
    MessagesPlaceholder(variable_name='history'),   # 세션별 이전 대화 메시지들이 들어갈 자리
    ('human', '{question}')
])
llm = init_chat_model('openai:gpt-4.1-mini')

chain = prompt | llm

# 히스토리 기능을 chain에 래핑
chain_with_history = RunnableWithMessageHistory(
    chain,                          # 실제로 싱행할 체인
    get_by_session_id,              # session_id로 히스토리 객체를 가져오는 함수
    input_messages_key='question',  # 입력 dict에서 'question'은 사용자 메시지로 인식
    history_messages_key='history'  # 프롬프트에서 히스토리를 받을 변수명
)

# 대화내역 쌓기
chain_with_history.invoke({
    'domain' : 'math',
    'question' : '우빈이는 강아지를 3마리 키우고 있습니다.'
}, config = {   # RunnableWithMessageHistory 설정
    'configurable':{
        'session_id':100    # 어떤 세션 히스토리를 사용할지 지정
    }
})

AIMessage(content='네, 우빈이가 강아지를 3마리 키우고 있다고 다시 한 번 알려주셨네요. 이와 관련해서 수학 문제나 궁금한 점이 있으면 말씀해 주세요! 도움이 될 수 있도록 하겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 109, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_e01c6f58e1', 'id': 'chatcmpl-D5mknlFZNZsUBAnSFplVWR740lCe3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c66-3341-7d41-9136-c076e35c16b9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 50, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
# 대화내역 쌓기
chain_with_history.invoke({
    'domain' : 'math',
    'question' : '소라는 고양이를 4마리 키우고 있습니다.'
}, config = {   # RunnableWithMessageHistory 설정
    'configurable':{
        'session_id':100    # 어떤 세션 히스토리를 사용할지 지정
    }
})

AIMessage(content='우빈이는 강아지 3마리, 소라는 고양이 4마리를 키우고 있네요. 두 사람이 키우는 동물의 총 마릿수를 구해볼까요?\n\n우빈이의 강아지 3마리 + 소라의 고양이 4마리 = 7마리\n\n총 7마리를 키우고 있습니다. 더 궁금한 점이 있으면 알려주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 181, 'total_tokens': 273, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mrURJaInGriBQgxZOU9rPZVtvDR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c6c-8514-7a12-ab3b-4a511f20d83e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 181, 'output_tokens': 92, 'total_tokens': 273, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reas

In [16]:
store

{'1': InMemoryHistory(messages=[[AIMessage(content='반갑습니다. 홍길동님', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])], [HumanMessage(content='그래 나 홍길동이야~ 반갑다!', additional_kwargs={}, response_metadata={})]]),
 '2': InMemoryHistory(messages=[]),
 100: InMemoryHistory(messages=[HumanMessage(content='우빈이는 강앚를 3마리 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='우빈이가 강아지 3마리를 키우고 있군요! 강아지와 관련해서 수학적으로 궁금한 점이 있나요? 아니면 다른 도움이 필요하신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 41, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_e01c6f58e1', 'id': 'chatcmpl-D5mY7T5w4BFQUiEb1Bu63gGtaOyA7', 'service_

## ChatMessageHistory

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:          # session_id로 해당 대화기록 객체를 가져오는 함수
    """session_id에 해당하는 대화내역 객체를 반환(없으면 생성)"""
    if session_id not in store:                                            # 아직 해당 세션 기록이 없으면
        store[session_id] = InMemoryHistory()                              # 새 InMemoryHistory를 만들어 store에 저장
    return store[session_id]                                               # 해당 세션의 대화기록 객체 반환

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 {domain}분야의 전문가 챗봇입니다."),                   # 시스템 역할: 도메인(분야) 전문가로 세팅
    MessagesPlaceholder(variable_name='history'),                           # 여기에 세션별 대화 내역이 자동으로 끼워짐
    ("human", "{question}")                                                 # 사용자의 현재 질문(입력 변수: question)
])

llm = init_chat_model("openai:gpt-4.1-mini")                               # OpenAI gpt-4.1-mini 모델 사용

chain = prompt | llm                                                       # prompt에 변수 채우고, 그 결과를 llm에 전달

chain_with_history = RunnableWithMessageHistory(
    chain,                                                                  # 히스토리를 붙일 원본 체인
    get_by_session_id,                                                      # session_id로 히스토리 객체를 가져오는 함수(없으면 생성)
    input_messages_key='question',                                          # 입력 딕셔너리에서 "사용자 메시지"로 취급할 키
    history_messages_key='history'                                          # 프롬프트의 MessagesPlaceholder 변수명과 동일해야 함
)

chain_with_history.invoke(
    {
        "domain": "심리상담",                                                  # system 프롬프트의 {domain}에 들어갈 값
        "question": "난 신승훈이야 반갑다"               # human 프롬프트의 {question}에 들어갈 값
    },
    config={                                                               # RunnableWithMessageHistory 설정
        "configurable": {
            "session_id": 200                                              # 어떤 세션의 대화기록을 쓸지 지정(= store[100]에 쌓임)
        }
    }
)

AIMessage(content='반갑습니다, 신승훈님! 심리 상담이나 마음에 관한 어떤 이야기든 편하게 나누고 싶으시면 말씀해 주세요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 37, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_e01c6f58e1', 'id': 'chatcmpl-D5mzDJVnUPSBv4srmC2WZwFNSGxpo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c73-d44d-75f1-bc6e-0da09d586bb7-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 38, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
chain_with_history.invoke(
    {
        "domain": "심리상담",                                                  # system 프롬프트의 {domain}에 들어갈 값
        "question": "요즘 날씨가 안좋아서 기분이 별로 좋지 않아!"               # human 프롬프트의 {question}에 들어갈 값
    },
    config={                                                               # RunnableWithMessageHistory 설정
        "configurable": {
            "session_id": 200                                              # 어떤 세션의 대화기록을 쓸지 지정(= store[100]에 쌓임)
        }
    }
)

AIMessage(content='날씨가 우중충하면 기분도 영향을 받기 쉽죠. 그런 날에는 자신을 조금 더 따뜻하게 돌보는 게 중요해요. 평소에 좋아하는 활동이나 휴식을 취하면서 기분 전환을 해보는 건 어떨까요? 혹시 기분을 좀 더 좋게 만들고 싶은 구체적인 방법이 궁금하다면 알려드릴게요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 100, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_e01c6f58e1', 'id': 'chatcmpl-D5mzEplteNhdLVadJsBDYC9lG925E', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c73-dc4d-7583-9db9-f6cdf1fb8510-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 100, 'output_tokens': 85, 'total_tokens': 185, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, '

In [19]:
store

{200: InMemoryHistory(messages=[HumanMessage(content='난 신승훈이야 반갑다', additional_kwargs={}, response_metadata={}), AIMessage(content='반갑습니다, 신승훈님! 심리 상담이나 마음에 관한 어떤 이야기든 편하게 나누고 싶으시면 말씀해 주세요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 37, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_e01c6f58e1', 'id': 'chatcmpl-D5mzDJVnUPSBv4srmC2WZwFNSGxpo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c73-d44d-75f1-bc6e-0da09d586bb7-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 38, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

##### 세션(메모리) 방식의 문제점
- 메모리 저장이라 영속성이 없음
    - 서버 재시작/재배포 하면 store가 날아가서 히스토리도 같이 사라짐
- 세션 식별이 끊기기 쉬움
    - 쿠키/세션ID가 유지되지 않으면 같은 사람인지 매칭이 안 됨
- 스케일 아웃(서버 여러 대)에서 깨짐
    - A서버 메모리에 저장된 히스토리를 B서버는 모름 → 대화가 끊김

그래서 보통 이렇게 구성한다.
- SQLite/Redis/RDB 같은 저장소에 대화 내역을 저장해서
    - 사용자가 재접속해도 user_id 또는 thread_id로 복원
- 프롬프트에는 보통
    - 최근 N턴 + 요약 형태로 넣어서 비용/토큰도 관리